INSTALACIÓN DE LIBRERIAS

In [1]:
pip install torch

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
pip install unidecode

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
pip install torch==1.7.1+cu110 torchvision==0.8.2+cu110 torchaudio==0.7.2 -f https://download.pytorch.org/whl/torch_stable.html


Looking in links: https://download.pytorch.org/whl/torch_stable.html
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement torch==1.7.1+cu110 (from versions: 2.0.0, 2.0.0+cpu, 2.0.0+cu117, 2.0.0+cu118, 2.0.1, 2.0.1+cpu, 2.0.1+cu117, 2.0.1+cu118, 2.1.0, 2.1.0+cpu, 2.1.0+cu118, 2.1.0+cu121)
ERROR: No matching distribution found for torch==1.7.1+cu110

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


IMPORTACIÓN DE LIBRERIAS

In [4]:
import pyodbc
import torch
from scipy.spatial.distance import cosine
import pandas as pd
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity
from fuzzywuzzy import fuzz
from unidecode import unidecode
from datetime import datetime
from pymongo import MongoClient
import re
import pyodbc
import numpy as np

c:\Users\jquispe\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Conexión BDD RELACIONAL

In [5]:
# Establece la cadena de conexión a tu servidor SQL Server
server = 'BFICHASIST23'
database = 'laboratorio'
username = 'sa'
password = '1719895086'

conn = pyodbc.connect(f'DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password};TIMEOUT=30')

In [6]:
# Ejecutamos el query para realizar la consulta a la base
base_med = "SELECT id_medico, apellido_paterno, apellido_materno, nombres, numero_identificacion, id_especialidad FROM dbo.medicos"

#  Se guarda en un df 
sql_data = pd.read_sql_query(base_med, conn)

#Se guarda el df en una nueva versión para no afectar la primera tabla
sql_data2 = sql_data

#Se concatena el nombre y apellidos para poder ejecutar el modelo
sql_data2['nombre_completo_sql'] = sql_data['apellido_paterno'] + ' ' + sql_data['apellido_materno'] + ' ' + sql_data['nombres']

# Filtrar las columnas con las que se va a trabajar
sql_data2 = sql_data2.iloc[:,[0,6,5,4]]

# Visualización del df
sql_data2


C:\Users\jquispe\AppData\Local\Temp\ipykernel_12404\3287920264.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  sql_data = pd.read_sql_query(base_med, conn)


,id_medico,nombre_completo_sql,id_especialidad,numero_identificacion
0,1,ROMERO HERRERA DENISE ELCIRA,6,0920116530
1,2,FLORES YANANGOMEZ DIANA ALEXANDRA,11,1103532808
2,3,NAVEDA PEREZ DIANA MARICELA,4,1804125001
3,4,COELLO AGUILAR DIANA MARLENE,15,0702216052
4,5,JUAREZ CARPIO DIEGO EDMUNDO,2,0701072472
...,...,...,...,...
3105,3106,RODRIGUEZ CEVALLOS JESSICA JASMIN,30,0703585331
3106,3107,ESCOBAR MURILLO JESSICA ELIZABETH,21,0603229337
3107,3108,GUEVARA SEVILLANO JESSICA CECIBEL,42,0802056796
3108,3109,GUILLEN GARCIA JESSICA ASUNCION,32,1309208393


In [ ]:
# Ejecutamos el query para realizar la consulta a la base
especialidad = "select * from dbo.especialidades"
# Se guarda en un df 
especialidad = pd.read_sql_query(especialidad, conn)
especialidad.head()

C:\Users\jquispe\AppData\Local\Temp\ipykernel_15832\690554618.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  especialidad = pd.read_sql_query(especialidad, conn)


,id_especialidad,nombre_especialidad
0,1,ALERGOLOGIA
1,2,CARDIOLOGIA
2,3,CIRUGIA CARDIOVASCULAR
3,4,CIRUGIA GENERAL
4,5,CIRUGIA ORAL Y MAXILOFACIAL


CARGA DE DATOS MEDICOS NUEVOS

In [ ]:
# Lee el archivo Excel en un DataFrame
med_nuevos = pd.read_excel('C:/Users/jquispe/Desktop/JACQUE/MAESTRÍA/tesis/NLP/INSUMO/MEDICOS NUEVOS1.xlsx')
#Para no afectar la tabla inicial generamos una copia
med_nuevos2 = med_nuevos
#Seleccionamos las columnas con las que se va a trabajar
med_nuevos2 = med_nuevos2.iloc[:, :-6].copy()
#Se concatena el nombre y apellidos para poder ejecutar el modelo
med_nuevos2['nombre_completo_exc'] = med_nuevos2['APELLIDO.PAT.'] + ' ' + med_nuevos2['APELLIDO.MAT.'] + ' ' + med_nuevos2['NOMBRES']
med_nuevos2.head()

,CODCUP,APELLIDO.PAT.,APELLIDO.MAT.,NOMBRES,CI,FEC_NAC,EMAIL,TEL,Q_FICO,nombre_especialidad,DIRECCION,nombre_completo_exc
0,21646453,R@MERO,#ERRERA,DENISSE,1719895087,1976-09-30,docjqv@gmail.com,979012860.0,4.0,Traumatología y Ortopedia,AV. 25 DE JULIO HOSPITAL TEODORO MALDONADO CARBO,R@MERO #ERRERA DENISSE
1,21646453,herrera,romero,ELCIRA denisse,916536758,1976-09-30,docjqv@gmail.com,979012860.0,4.0,Traumatología y Ortopedia,AV. 25 DE JULIO HOSPITAL TEODORO MALDONADO CARBO,herrera romero ELCIRA denisse
2,60731411,RODRIGUEZ,CEVALLO,JESIKA JAZMIN,704785880,NaT,jcorellanaroman@hotmail.com,NaN,4.0,Medicina General,AV. 8 DE ABRIL Y TELMO TEJADA,RODRIGUEZ CEVALLO JESIKA JAZMIN
3,60731411,ORELLANA,ROMAN,JUAN CARLOS,704785880,NaT,jcorellanaroman@hotmail.com,NaN,4.0,Medicina General,AV. 8 DE ABRIL Y TELMO TEJADA,ORELLANA ROMAN JUAN CARLOS
4,60731411,ROMAN,ORELLANA,CARLOS JUAN,704785880,NaT,jcorellanaroman@hotmail.com,NaN,4.0,Medicina General,AV. 8 DE ABRIL Y TELMO TEJADA,ROMAN ORELLANA CARLOS JUAN


In [ ]:
#Visualización Tipo de Datos de las variables del archivo cargado
med_nuevos2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   CODCUP               14 non-null     int64         
 1   APELLIDO.PAT.        14 non-null     object        
 2   APELLIDO.MAT.        14 non-null     object        
 3   NOMBRES              14 non-null     object        
 4   CI                   14 non-null     int64         
 5   FEC_NAC              5 non-null      datetime64[ns]
 6   EMAIL                11 non-null     object        
 7   TEL                  5 non-null      float64       
 8   Q_FICO               11 non-null     float64       
 9   nombre_especialidad  14 non-null     object        
 10  DIRECCION            14 non-null     object        
 11  nombre_completo_exc  14 non-null     object        
dtypes: datetime64[ns](1), float64(2), int64(2), object(7)
memory usage: 1.4+ KB


TRATAMIENTO DE DATOS

In [ ]:
# Función para eliminar tildes y convertir "ñ" en "n" en una cadena
def quitar_tildes_y_n(cadena):
    if isinstance(cadena, str):
        cadena_sin_tildes = unidecode(cadena)
        cadena_sin_n = cadena_sin_tildes.replace('ñ', 'n')
        return cadena_sin_n
    else:
        return cadena
    
# Aplicar la función eliminar tildes y ñ solo a las columnas de tipo objeto (texto)
for columna in med_nuevos2.select_dtypes(include=['object']).columns:
    med_nuevos2[columna] = med_nuevos2[columna].apply(quitar_tildes_y_n) # esta funcion quita las tildes y las ñ

# Eliminación de espacios en blanco
med_nuevos2[columna] = med_nuevos2[columna].str.strip()

# elimina las filas duplicadas 
med_nuevos2 = med_nuevos2.drop_duplicates() 

# Convierte en tipo texto
med_nuevos2['TEL'] = med_nuevos2['TEL'].apply(lambda x: str(x).replace('.0', ''))
med_nuevos2['Q_FICO'] = med_nuevos2['Q_FICO'].apply(lambda x: str(x).replace('.0', ''))
med_nuevos2['CI'] = med_nuevos2['CI'].astype(str)

# Función para que la cedula y celular tenga la estructura de 10 digitos
def convertir_a_texto(valor):
    if isinstance(valor, str):
        valor_texto = str(valor)
        if len(valor_texto) == 9:
            valor_texto = '0' + valor_texto
        return valor_texto
    else:
        return str(valor)
    
# Aplicar la función solo a las columnas "ci" y "celular"
med_nuevos2['TEL'] = med_nuevos2['TEL'].apply(convertir_a_texto)
med_nuevos2['CI'] = med_nuevos2['CI'].apply(convertir_a_texto)
    

# Función para eliminar caracteres especiales de una cadena
def eliminar_caracteres_especiales(cadena):
    if isinstance(cadena, str):
        return re.sub(r'[^\w\s]', '', cadena)
    else:
        return cadena

# Obtener las columnas de texto (excepto la columna de correo)
columnas_texto = [col for col in med_nuevos2.select_dtypes(include=['object']).columns if col != 'EMAIL']

# Iterar a través de las columnas de texto y aplicar la función
for columna in columnas_texto:
    med_nuevos2[columna] = med_nuevos2[columna].apply(eliminar_caracteres_especiales)

# convierte en mayusculas absolutamente todo el texto 
med_nuevos2 = med_nuevos2.applymap(lambda x: x.upper() if isinstance(x, str) else x) 


# Visualizar los cambios
med_nuevos2

,CODCUP,APELLIDO.PAT.,APELLIDO.MAT.,NOMBRES,CI,FEC_NAC,EMAIL,TEL,Q_FICO,nombre_especialidad,DIRECCION,nombre_completo_exc
0,21646453,RMERO,ERRERA,DENISSE,1719895087,1976-09-30,DOCJQV@GMAIL.COM,0979012860,4,TRAUMATOLOGIA Y ORTOPEDIA,AV 25 DE JULIO HOSPITAL TEODORO MALDONADO CARBO,RMERO ERRERA DENISSE
1,21646453,HERRERA,ROMERO,ELCIRA DENISSE,0916536758,1976-09-30,DOCJQV@GMAIL.COM,0979012860,4,TRAUMATOLOGIA Y ORTOPEDIA,AV 25 DE JULIO HOSPITAL TEODORO MALDONADO CARBO,HERRERA ROMERO ELCIRA DENISSE
2,60731411,RODRIGUEZ,CEVALLO,JESIKA JAZMIN,0704785880,NaT,JCORELLANAROMAN@HOTMAIL.COM,NAN,4,MEDICINA GENERAL,AV 8 DE ABRIL Y TELMO TEJADA,RODRIGUEZ CEVALLO JESIKA JAZMIN
3,60731411,ORELLANA,ROMAN,JUAN CARLOS,0704785880,NaT,JCORELLANAROMAN@HOTMAIL.COM,NAN,4,MEDICINA GENERAL,AV 8 DE ABRIL Y TELMO TEJADA,ORELLANA ROMAN JUAN CARLOS
4,60731411,ROMAN,ORELLANA,CARLOS JUAN,0704785880,NaT,JCORELLANAROMAN@HOTMAIL.COM,NAN,4,MEDICINA GENERAL,AV 8 DE ABRIL Y TELMO TEJADA,ROMAN ORELLANA CARLOS JUAN
8,61595738,PAZMINO,PALACIOS,JOSE LUIS,0104153945,NaT,NaN,0987671233,4,PEDIATRIA,MARCEL LANIADO 1028 Y PAEZ,PAZMINO PALACIOS JOSE LUIS
11,61598169,VINTIMILLA,SARMIENTO,MARIA LUISA,1713838876,1975-02-22,MALU.VS.AP@HOTMAIL.COM,NAN,NAN,MEDICINA GENERAL,PRAXMED VALLE DE LOS CHILLOS,VINTIMILLA SARMIENTO MARIA LUISA


In [ ]:
#Fecha de nacimiento a texto y con el formato adecuado para cargar a las bases
med_nuevos2['FEC_NAC'] = med_nuevos2['FEC_NAC'].dt.strftime('%Y/%m/%d')

med_nuevos2


,CODCUP,APELLIDO.PAT.,APELLIDO.MAT.,NOMBRES,CI,FEC_NAC,EMAIL,TEL,Q_FICO,nombre_especialidad,DIRECCION,nombre_completo_exc
0,21646453,RMERO,ERRERA,DENISSE,1719895087,1976/09/30,DOCJQV@GMAIL.COM,0979012860,4,TRAUMATOLOGIA Y ORTOPEDIA,AV 25 DE JULIO HOSPITAL TEODORO MALDONADO CARBO,RMERO ERRERA DENISSE
1,21646453,HERRERA,ROMERO,ELCIRA DENISSE,0916536758,1976/09/30,DOCJQV@GMAIL.COM,0979012860,4,TRAUMATOLOGIA Y ORTOPEDIA,AV 25 DE JULIO HOSPITAL TEODORO MALDONADO CARBO,HERRERA ROMERO ELCIRA DENISSE
2,60731411,RODRIGUEZ,CEVALLO,JESIKA JAZMIN,0704785880,NaN,JCORELLANAROMAN@HOTMAIL.COM,NAN,4,MEDICINA GENERAL,AV 8 DE ABRIL Y TELMO TEJADA,RODRIGUEZ CEVALLO JESIKA JAZMIN
3,60731411,ORELLANA,ROMAN,JUAN CARLOS,0704785880,NaN,JCORELLANAROMAN@HOTMAIL.COM,NAN,4,MEDICINA GENERAL,AV 8 DE ABRIL Y TELMO TEJADA,ORELLANA ROMAN JUAN CARLOS
4,60731411,ROMAN,ORELLANA,CARLOS JUAN,0704785880,NaN,JCORELLANAROMAN@HOTMAIL.COM,NAN,4,MEDICINA GENERAL,AV 8 DE ABRIL Y TELMO TEJADA,ROMAN ORELLANA CARLOS JUAN
8,61595738,PAZMINO,PALACIOS,JOSE LUIS,0104153945,NaN,NaN,0987671233,4,PEDIATRIA,MARCEL LANIADO 1028 Y PAEZ,PAZMINO PALACIOS JOSE LUIS
11,61598169,VINTIMILLA,SARMIENTO,MARIA LUISA,1713838876,1975/02/22,MALU.VS.AP@HOTMAIL.COM,NAN,NAN,MEDICINA GENERAL,PRAXMED VALLE DE LOS CHILLOS,VINTIMILLA SARMIENTO MARIA LUISA


In [ ]:
# RENOMBRAR LAS COLUMNAS QUE TIENEN PUNTO
med_nuevos2.rename(columns={'APELLIDO.PAT.': 'APELLIDO_PAT'}, inplace=True)
med_nuevos2.rename(columns={'APELLIDO.MAT.': 'APELLIDO_MAT'}, inplace=True)

med_nuevos2


,CODCUP,APELLIDO_PAT,APELLIDO_MAT,NOMBRES,CI,FEC_NAC,EMAIL,TEL,Q_FICO,nombre_especialidad,DIRECCION,nombre_completo_exc
0,21646453,RMERO,ERRERA,DENISSE,1719895087,1976/09/30,DOCJQV@GMAIL.COM,0979012860,4,TRAUMATOLOGIA Y ORTOPEDIA,AV 25 DE JULIO HOSPITAL TEODORO MALDONADO CARBO,RMERO ERRERA DENISSE
1,21646453,HERRERA,ROMERO,ELCIRA DENISSE,0916536758,1976/09/30,DOCJQV@GMAIL.COM,0979012860,4,TRAUMATOLOGIA Y ORTOPEDIA,AV 25 DE JULIO HOSPITAL TEODORO MALDONADO CARBO,HERRERA ROMERO ELCIRA DENISSE
2,60731411,RODRIGUEZ,CEVALLO,JESIKA JAZMIN,0704785880,NaN,JCORELLANAROMAN@HOTMAIL.COM,NAN,4,MEDICINA GENERAL,AV 8 DE ABRIL Y TELMO TEJADA,RODRIGUEZ CEVALLO JESIKA JAZMIN
3,60731411,ORELLANA,ROMAN,JUAN CARLOS,0704785880,NaN,JCORELLANAROMAN@HOTMAIL.COM,NAN,4,MEDICINA GENERAL,AV 8 DE ABRIL Y TELMO TEJADA,ORELLANA ROMAN JUAN CARLOS
4,60731411,ROMAN,ORELLANA,CARLOS JUAN,0704785880,NaN,JCORELLANAROMAN@HOTMAIL.COM,NAN,4,MEDICINA GENERAL,AV 8 DE ABRIL Y TELMO TEJADA,ROMAN ORELLANA CARLOS JUAN
8,61595738,PAZMINO,PALACIOS,JOSE LUIS,0104153945,NaN,NaN,0987671233,4,PEDIATRIA,MARCEL LANIADO 1028 Y PAEZ,PAZMINO PALACIOS JOSE LUIS
11,61598169,VINTIMILLA,SARMIENTO,MARIA LUISA,1713838876,1975/02/22,MALU.VS.AP@HOTMAIL.COM,NAN,NAN,MEDICINA GENERAL,PRAXMED VALLE DE LOS CHILLOS,VINTIMILLA SARMIENTO MARIA LUISA


In [ ]:
# Realiza la fusión (merge) de los DataFrames para obtener el código de especialidad
med_nuevos2 = med_nuevos2.merge(especialidad, on='nombre_especialidad', how='left')
# Renombra la columna 'Codigo' a 'Especialidad'
med_nuevos2 = med_nuevos2.rename(columns={'id_especialidad': 'cod_Espe'})
# Eliminar una columna específica del DataFrame
med_nuevos2 = med_nuevos2.drop('nombre_especialidad', axis=1)
med_nuevos2

,CODCUP,APELLIDO_PAT,APELLIDO_MAT,NOMBRES,CI,FEC_NAC,EMAIL,TEL,Q_FICO,DIRECCION,nombre_completo_exc,cod_Espe
0,21646453,RMERO,ERRERA,DENISSE,1719895087,1976/09/30,DOCJQV@GMAIL.COM,0979012860,4,AV 25 DE JULIO HOSPITAL TEODORO MALDONADO CARBO,RMERO ERRERA DENISSE,42
1,21646453,HERRERA,ROMERO,ELCIRA DENISSE,0916536758,1976/09/30,DOCJQV@GMAIL.COM,0979012860,4,AV 25 DE JULIO HOSPITAL TEODORO MALDONADO CARBO,HERRERA ROMERO ELCIRA DENISSE,42
2,60731411,RODRIGUEZ,CEVALLO,JESIKA JAZMIN,0704785880,NaN,JCORELLANAROMAN@HOTMAIL.COM,NAN,4,AV 8 DE ABRIL Y TELMO TEJADA,RODRIGUEZ CEVALLO JESIKA JAZMIN,23
3,60731411,ORELLANA,ROMAN,JUAN CARLOS,0704785880,NaN,JCORELLANAROMAN@HOTMAIL.COM,NAN,4,AV 8 DE ABRIL Y TELMO TEJADA,ORELLANA ROMAN JUAN CARLOS,23
4,60731411,ROMAN,ORELLANA,CARLOS JUAN,0704785880,NaN,JCORELLANAROMAN@HOTMAIL.COM,NAN,4,AV 8 DE ABRIL Y TELMO TEJADA,ROMAN ORELLANA CARLOS JUAN,23
5,61595738,PAZMINO,PALACIOS,JOSE LUIS,0104153945,NaN,NaN,0987671233,4,MARCEL LANIADO 1028 Y PAEZ,PAZMINO PALACIOS JOSE LUIS,37
6,61598169,VINTIMILLA,SARMIENTO,MARIA LUISA,1713838876,1975/02/22,MALU.VS.AP@HOTMAIL.COM,NAN,NAN,PRAXMED VALLE DE LOS CHILLOS,VINTIMILLA SARMIENTO MARIA LUISA,23


Algoritmo de verificación de cédulas

In [ ]:
def validar_cedula(cedula):
    # Verificar que la cédula tenga 10 dígitos
    if len(cedula) != 10:
        return False
    
    # Verificar que los primeros dos dígitos sean válidos
    provincia = int(cedula[:2])
    if provincia < 1 or provincia > 24:
        return False
    
    # Verificar el tercer dígito (dígito de control)
    tercer_digito = int(cedula[2])
    if tercer_digito < 0 or tercer_digito > 5:
        return False
    
    # Verificar el último dígito (dígito de verificación)
    verificacion = int(cedula[9])
    
    # Calcular el dígito de verificación esperado
    coeficientes = [2, 1, 2, 1, 2, 1, 2, 1, 2]
    suma = 0
    for i in range(9):
        digito = int(cedula[i])
        producto = digito * coeficientes[i]
        if producto > 9:
            producto -= 9
        suma += producto
    
    verificacion_esperada = 10 - (suma % 10)
    if verificacion_esperada == 10:
        verificacion_esperada = 0
    
    # Comparar el dígito de verificación calculado con el proporcionado
    return verificacion == verificacion_esperada

# Aplicacion de la funcion
med_nuevos2['verif_ced'] = med_nuevos2['CI'].apply(validar_cedula)
med_nuevos2


,CODCUP,APELLIDO_PAT,APELLIDO_MAT,NOMBRES,CI,FEC_NAC,EMAIL,TEL,Q_FICO,DIRECCION,nombre_completo_exc,cod_Espe,verif_ced
0,21646453,RMERO,ERRERA,DENISSE,1719895087,1976/09/30,DOCJQV@GMAIL.COM,0979012860,4,AV 25 DE JULIO HOSPITAL TEODORO MALDONADO CARBO,RMERO ERRERA DENISSE,42,False
1,21646453,HERRERA,ROMERO,ELCIRA DENISSE,0916536758,1976/09/30,DOCJQV@GMAIL.COM,0979012860,4,AV 25 DE JULIO HOSPITAL TEODORO MALDONADO CARBO,HERRERA ROMERO ELCIRA DENISSE,42,True
2,60731411,RODRIGUEZ,CEVALLO,JESIKA JAZMIN,0704785880,NaN,JCORELLANAROMAN@HOTMAIL.COM,NAN,4,AV 8 DE ABRIL Y TELMO TEJADA,RODRIGUEZ CEVALLO JESIKA JAZMIN,23,True
3,60731411,ORELLANA,ROMAN,JUAN CARLOS,0704785880,NaN,JCORELLANAROMAN@HOTMAIL.COM,NAN,4,AV 8 DE ABRIL Y TELMO TEJADA,ORELLANA ROMAN JUAN CARLOS,23,True
4,60731411,ROMAN,ORELLANA,CARLOS JUAN,0704785880,NaN,JCORELLANAROMAN@HOTMAIL.COM,NAN,4,AV 8 DE ABRIL Y TELMO TEJADA,ROMAN ORELLANA CARLOS JUAN,23,True
5,61595738,PAZMINO,PALACIOS,JOSE LUIS,0104153945,NaN,NaN,0987671233,4,MARCEL LANIADO 1028 Y PAEZ,PAZMINO PALACIOS JOSE LUIS,37,True
6,61598169,VINTIMILLA,SARMIENTO,MARIA LUISA,1713838876,1975/02/22,MALU.VS.AP@HOTMAIL.COM,NAN,NAN,PRAXMED VALLE DE LOS CHILLOS,VINTIMILLA SARMIENTO MARIA LUISA,23,True


MODELO DE DETECCION DE DUPLICADOS

MODELO FUZZY TOKEN SORT RATIO

In [ ]:
# Crear una lista para almacenar las coincidencias
coincidencias = []

# Iterar a través de los nombres en el archivo Excel
for index_excel, row_excel in med_nuevos2.iterrows():
    nombre_excel = row_excel['nombre_completo_exc']
    mejor_coincidencia = None
    mejor_porcentaje = 0

    for index_db, row_db in sql_data2.iterrows():
        nombre_db = row_db['nombre_completo_sql']

        # Calcular la similitud entre los nombres y apellidos
        similitud_nombre = fuzz.token_sort_ratio(nombre_excel.lower(), nombre_db.lower())

        # Si la similitud está por encima del umbral y es mejor que la anterior, actualizar la mejor coincidencia
        if similitud_nombre > mejor_porcentaje:
            mejor_coincidencia = pd.concat([row_excel, row_db])
            mejor_coincidencia['Porcentaje de Similitud'] = similitud_nombre
            mejor_porcentaje = similitud_nombre

    # Agregar la mejor coincidencia a la lista de coincidencias
    if mejor_coincidencia is not None:
        coincidencias.append(mejor_coincidencia)

# Crear un DataFrame con las mejores coincidencias
coincidencias_df = pd.DataFrame(coincidencias)

# Función para asignar "Medico en Base" o "Medico Nuevo" en función de la similitud
def asignar_medico(row):
    return "MEDICO EN BASE" if row['Porcentaje de Similitud'] > 79 else "MEDICO NUEVO"

# Aplicar la función a cada fila y crear una nueva columna "OBSERVACION"
coincidencias_df['OBSERVACION'] = coincidencias_df.apply(asignar_medico, axis=1)

# Visualización de resultados
coincidencias_df

,CODCUP,APELLIDO_PAT,APELLIDO_MAT,NOMBRES,CI,FEC_NAC,EMAIL,TEL,Q_FICO,DIRECCION,nombre_completo_exc,cod_Espe,verif_ced,id_medico,nombre_completo_sql,id_especialidad,numero_identificacion,Porcentaje de Similitud,OBSERVACION
0,21646453,RMERO,ERRERA,DENISSE,1719895087,30/09/1976,DOCJQV@GMAIL.COM,0979012860,4,AV 25 DE JULIO HOSPITAL TEODORO MALDONADO CARBO,RMERO ERRERA DENISSE,42,False,1,ROMERO HERRERA DENISE ELCIRA,6,0920116530,79,MEDICO NUEVO
Unnamed 0,21646453,HERRERA,ROMERO,ELCIRA DENISSE,0916536758,30/09/1976,DOCJQV@GMAIL.COM,0979012860,4,AV 25 DE JULIO HOSPITAL TEODORO MALDONADO CARBO,HERRERA ROMERO ELCIRA DENISSE,42,True,1,ROMERO HERRERA DENISE ELCIRA,6,0920116530,98,MEDICO EN BASE
Unnamed 1,60731411,RODRIGUEZ,CEVALLO,JESIKA JAZMIN,0704785880,NaN,JCORELLANAROMAN@HOTMAIL.COM,NAN,4,AV 8 DE ABRIL Y TELMO TEJADA,RODRIGUEZ CEVALLO JESIKA JAZMIN,23,True,3106,RODRIGUEZ CEVALLOS JESSICA JASMIN,30,0703585331,91,MEDICO EN BASE
Unnamed 2,60731411,ORELLANA,ROMAN,JUAN CARLOS,0704785880,NaN,JCORELLANAROMAN@HOTMAIL.COM,NAN,4,AV 8 DE ABRIL Y TELMO TEJADA,ORELLANA ROMAN JUAN CARLOS,23,True,3044,ROMERO RODRIGUEZ JUAN CARLOS,10,1205075557,70,MEDICO NUEVO
Unnamed 3,60731411,ROMAN,ORELLANA,CARLOS JUAN,0704785880,NaN,JCORELLANAROMAN@HOTMAIL.COM,NAN,4,AV 8 DE ABRIL Y TELMO TEJADA,ROMAN ORELLANA CARLOS JUAN,23,True,3044,ROMERO RODRIGUEZ JUAN CARLOS,10,1205075557,70,MEDICO NUEVO
Unnamed 4,61595738,PAZMINO,PALACIOS,JOSE LUIS,0104153945,NaN,NaN,0987671233,4,MARCEL LANIADO 1028 Y PAEZ,PAZMINO PALACIOS JOSE LUIS,37,True,2922,PALMA PALACIOS LUPE AZUCENA,16,1301974091,64,MEDICO NUEVO
Unnamed 5,61598169,VINTIMILLA,SARMIENTO,MARIA LUISA,1713838876,22/02/1975,MALU.VS.AP@HOTMAIL.COM,NAN,NAN,PRAXMED VALLE DE LOS CHILLOS,VINTIMILLA SARMIENTO MARIA LUISA,23,True,992,AGUILAR VINTIMILLA MIRIAN ERMINIA,35,1802271716,68,MEDICO NUEVO


Conexion con MONGODB para guardar historicos

In [ ]:
# Conexión al servidor MongoDB local
client = MongoClient("localhost", 27017)

# Accede a una base de datos
db = client.laboratorio

# Convierte el DataFrame en un diccionario
registros_historicos = coincidencias_df.to_dict(orient='records')

# Obtén la fecha actual
fecha_hoy = datetime.now()

# Agregar la fecha de hoy a cada registro en el diccionario
for registro in registros_historicos:
    registro['fecha_de_ejecucion'] = fecha_hoy

# Colección en la que se almacenarán los registros históricos
historico_collection = db.historicos

# Inserta los registros en la colección de historial
historico_collection.insert_many(registros_historicos)


MEDICOS NUEVOS CARGAR EN LA BASE

In [ ]:
# Filtrar información de médicos nuevos
carga_base = coincidencias_df[coincidencias_df['OBSERVACION'] == 'MEDICO NUEVO']

# Seleccionamos las columnas que se van a cargar
carga_base1 = carga_base.iloc[:, :12].copy()
carga_base1.head()

,CODCUP,APELLIDO_PAT,APELLIDO_MAT,NOMBRES,CI,FEC_NAC,EMAIL,TEL,Q_FICO,DIRECCION,nombre_completo_exc,cod_Espe
Unnamed 2,60731411,ORELLANA,ROMAN,JUAN CARLOS,0704785880,NaN,JCORELLANAROMAN@HOTMAIL.COM,NAN,4,AV 8 DE ABRIL Y TELMO TEJADA,ORELLANA ROMAN JUAN CARLOS,23
Unnamed 3,60731411,ROMAN,ORELLANA,CARLOS JUAN,0704785880,NaN,JCORELLANAROMAN@HOTMAIL.COM,NAN,4,AV 8 DE ABRIL Y TELMO TEJADA,ROMAN ORELLANA CARLOS JUAN,23
Unnamed 4,61595738,PAZMINO,PALACIOS,JOSE LUIS,0104153945,NaN,NaN,0987671233,4,MARCEL LANIADO 1028 Y PAEZ,PAZMINO PALACIOS JOSE LUIS,37
Unnamed 5,61598169,VINTIMILLA,SARMIENTO,MARIA LUISA,1713838876,22/02/1975,MALU.VS.AP@HOTMAIL.COM,NAN,NAN,PRAXMED VALLE DE LOS CHILLOS,VINTIMILLA SARMIENTO MARIA LUISA,23


Conexión SQL Server para inserción de información

In [ ]:


# Establece la cadena de conexión a tu servidor SQL Server
server = 'BFICHASIST23'
database = 'laboratorio'
username = 'sa'
password = '1719895086'

conn = pyodbc.connect(f'DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password};TIMEOUT=30')
cursor = conn.cursor()

for index, row in carga_base1.iterrows():
    apellido_paterno = row['APELLIDO_PAT'] if not pd.isnull(row['APELLIDO_PAT']) else None
    apellido_materno = row['APELLIDO_MAT'] if not pd.isnull(row['APELLIDO_MAT']) else None
    nombres = row['NOMBRES'] if not pd.isnull(row['NOMBRES']) else None
    id_tipo_identificacion = '1'
    numero_identificacion = row['CI']

    # Comprobación de la fecha de nacimiento
    if isinstance(row['FEC_NAC'], str):
        try:
            fecha_nacimiento = datetime.strptime(row['FEC_NAC'], '%Y/%m/%d').strftime('%Y-%m-%d')
        except ValueError as e:
            print(f"Error en la fila {index}: {e}")
            fecha_nacimiento = None
    else:
        fecha_nacimiento = row['FEC_NAC'].strftime('%Y-%m-%d') if not pd.isnull(row['FEC_NAC']) else None

    email = row['EMAIL'] if not pd.isnull(row['EMAIL']) else None
    celular = row['TEL'] if not pd.isnull(row['TEL']) else None
    direccion = row['DIRECCION'] if not pd.isnull(row['DIRECCION']) else None

    # Comprobación de id_especialidad
    id_especialidad = row['cod_Espe']
    if not pd.isnull(id_especialidad):
        id_especialidad = int(id_especialidad) if isinstance(id_especialidad, (int, float)) else 0
    else:
        id_especialidad = None

    # Ejecuta la consulta
    cursor.execute("INSERT INTO dbo.medicos (apellido_paterno, apellido_materno, nombres, id_tipo_identificacion, numero_identificacion, fecha_nacimiento, email, celular, direccion, id_especialidad) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)",
                   (apellido_paterno, apellido_materno, nombres, id_tipo_identificacion, numero_identificacion, fecha_nacimiento, email, celular, direccion, id_especialidad))

# Confirma la transacción y cierra la conexión
conn.commit()
conn.close()


Error en la fila Unnamed 5: time data '22/02/1975' does not match format '%Y/%m/%d'
